In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import csv

In [3]:
def get_preperation():
    merged_list = []
    skip_first = False # col name
    chef_file = '/input/recipe_details_27-12-2017.csv'
    with open(chef_file, 'r') as f:
        chefkoch = csv.reader(f)
        for row in chefkoch:
            if skip_first:
                skip_first = False
                continue
            try:
                merged_list.append(row[2])
            except: 
                continue
    text = ' '.join(merged_list[:6000])
    return(text)

In [4]:
text = get_preperation()

### Lnge des gesamten Zubereitungstextes von allen Rezepten

In [5]:
print("{:,} Zeichen fuer 20'000 Rezepte".format(len(text)))

4,247,529 Zeichen fuer 20'000 Rezepte


In [6]:
import os
os.environ["KERAS_BACKEND"] = "theano"
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
import keras; import keras.backend
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
if keras.backend.backend() != 'theano':
    raise BaseException("This script uses other backend")
else:
    keras.backend.set_image_dim_ordering('th')
    print("Backend ok")


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

2018-01-05 15:52:46,282 theano.sandbox.cuda WARNING  __init__.py:502 - use() The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


Backend ok


In [9]:
import random
import numpy as np
from glob import glob

chars = list(set(text))

# set a fixed vector size
max_len = 20

In [10]:
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(max_len, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 512)           1333248   
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 138)               70794     
_________________________________________________________________
activation_1 (Activation)    (None, 138)               0         
Total params: 3,503,242
Trainable params: 3,503,242
Non-trainable params: 0
_________________________________________________________________


In [12]:
step = 3
inputs = []
outputs = []
for i in range(0, len(text) - max_len, step):
    inputs.append(text[i:i+max_len])
    outputs.append(text[i+max_len])

In [11]:
! pip install psutil

In [13]:
import psutil
psutil.virtual_memory()

svmem(total=27394269184, available=25805017088, percent=5.8, used=2534232064, free=24860037120, active=1338621952, inactive=603824128, buffers=102944768, cached=842035200, shared=11718656)

In [14]:
char_labels = {ch:i for i, ch in enumerate(chars)}
labels_char = {i:ch for i, ch in enumerate(chars)}

# using bool to reduce memory usage
X = np.zeros((len(inputs), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(inputs), len(chars)), dtype=np.bool)

# one-hot vector
for i, example in enumerate(inputs):
    for t, char in enumerate(example):
        X[i, t, char_labels[char]] = 1
    y[i, char_labels[outputs[i]]] = 1

In [15]:
def generate(temperature=0.35, seed=None, num_chars=150):
    predicate=lambda x: len(x) < num_chars
    
    if seed is not None and len(seed) < max_len:
        raise Exception('{} chars long'.format(max_len))

    else:
        start_idx = random.randint(0, len(text) - max_len - 1)
        seed = text[start_idx:start_idx + max_len]

    sentence = seed
    generated = sentence

    while predicate(generated):
        # generate input tensor
        x = np.zeros((1, max_len, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_labels[char]] = 1.

        probs = model.predict(x, verbose=0)[0]
        next_idx = sample(probs, temperature)
        next_char = labels_char[next_idx]

        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

def sample(probs, temperature):
    a = np.log(probs)/temperature
    dist = np.exp(a)/np.sum(np.exp(a))
    choices = range(len(probs))
    return np.random.choice(choices, p=dist)

In [16]:
epochs = 100
nb_epoch_num = 0
for i in range(epochs):
    print('epoch %d'%i)

    model.fit(X, y, batch_size=128, epochs=1)
    nb_epoch_num += 1
    model.save_weights('/output/RNN_checkpoint_{}_epoch.hdf5'.format(nb_epoch_num))

    # preview
    for temp in [0.2, 0.5, 1., 1.2]:
        print('temperature: %0.2f'%temp)
        print('%s'%generate(temperature=temp))

epoch 0
Epoch 1/1
1415837/1415837 [==============================] - 2114s 1ms/step - loss: 1.2836
temperature: 0.20
ackpapier ausgelegte Scheibe schneiden.   Die Masse der Schmeckt auch ein Sieb auf dem Teig verschließen.   Die Masse der Schmeckt ist die Sahne auf d
temperature: 0.50
te einkochen lassen.   Die Masse zu eine Scheibe Schnee legen.   Den Teig in eine Schüssel geben und auf den Teig geben und unter Rühren auf der Papri
temperature: 1.00
inzufügen und einige Krümela sinde Zucchini und zu etwas Rotteler estrekte Masse oder ZimtCuetschmand werden, getrendten forme und die Vanillius Zucch
temperature: 1.20
 bestreichen.  Die Müblis oben sola halz verwendet ganz säftig mar einfach gießen. Dazu passt Bahler und Feigsalze putzen und dann das Fleisch wie Aho
epoch 1
Epoch 1/1
1415837/1415837 [==============================] - 2115s 1ms/step - loss: 0.9711
temperature: 0.20
rfel schneiden. Die Schokolade dazu geben und mit der Sahne und den Salz abschmecken.  Die Brote entfernen u

hini dazu und alles mit einem Klarkipfel vermengen und anschließend das Fleisch und Rosenkohl abschmecken.  Nach dem Auskühlen mischen und das Sauce k
temperature: 1.20
 die Frühlingszwiebeln gut abhacken. Aus Eismarkreis oder , wenn es gutes Geechmackzulatt für Im obige 16pmibläckeren. Das kaunte gerindt ist. Von dem
epoch 11
Epoch 1/1
 376064/1415837 [======>.......................] - ETA: 25:55 - loss: 0.8463

KeyboardInterrupt: 

In [21]:
print('%s' % generate(temperature=0.4,
                      seed='Ich salze meine Nudeln mit Salz',
                      num_chars=2000))

aten. Peperoni hinzugeben und dann die Zwiebeln und den Knoblauch schälen und in feine Streifen schneiden.  Die Mandeln mit der Sahne abschmecken.  Die Backzeit auf den Boden streuen. Die Teigstücke und die Kartoffeln und den Schnittlauch unterheben.   Die Paprika waschen und trocken tupfen.  Die Schalotten waschen, trocken tupfen und in Streifen schneiden. Die Mandeln und die Sahne unterrühren. Die Zwiebel in Streifen schneiden. Die Schalotten und den Mozzarella in den Topf geben und mit dem Backpulver vermischen. Das Ei geraspelt werden. Die Schokolade in Stücke schneiden.   Die Schalotten in kleine Streifen schneiden. Die Schalotte und die Paprika putzen, waschen, putzen und in Streifen schneiden. Die Kartoffeln schälen und in Streifen schneiden, das Fleisch auf den Boden geben und glatt streichen.  Die Sahne und die Mandeln dazugeben. Das Hackfleisch mit dem Saft der restlichen Walnüsse und den Schneebese vermischen. Die Schale beiseite stellen.   Die Masse in eine gefettete oder m

In [22]:
print('%s' % generate(temperature=1.0,
                      seed='Pfanne reinigen, kein Wasser dazu giessen',
                      num_chars=2000))

opf gegeben und erwärmen. Mit der Rosinen Kartoffeln schälen, fein hacken, die Brühe fein würfeln. Die Schalotten putzen, waschen und trockentupfen. Saft mit Salz und Pfeffer bestreichen und gut vermischen. Bei 200°C treilt den Mixer geben und abschmecken. Alles in etwas Öl erhitzen und das Fleisch sehr gutnliemurgrobenwürfel und die Zwiebeln würfeln und schälen. -Rikoten (auch Platten entsteht. auf Maraleflocker und Kirschen zusammen des Öl rollen und klein schneiden. Anschließend mit geschlossenen Mandarinen bestreuen. Die Kartoffeln kurz erneut verrühren, tunssieben.  Rucola in Scheiben schneiden und in die Hälfte des Karütichen darin unter Rühren braten.  Die Steherzwiebelwürfel und Schnittlauch ziemen, trocken schütteln, wenn er schnell ge anderem Cereme) auf den Topf verteilen. Kürbisfleisch waschen und in Kartoffeln bei Stück geben und glasig marinieren bzw. restliche Gurke auf dem Teller geben und verarbeiten. Heiß werden lassen. Kurz vor Kartoffel werden für eine Schüssel Wäss

In [23]:
print('%s' % generate(temperature=0.2,
seed='Alles dazugeben und in einen Kochtopf auf mittlerer Hitze kurz aufkochen lassen.',
                      num_chars=5000))

usstecher entfernen.   Die Mandeln in eine Schüssel geben und mit Salz und Pfeffer würzen. Die Schalotten und den Knoblauch schälen und in Streifen schneiden. Die Schalotten und den Schnittlauch in Streifen schneiden. Die Zwiebeln und die Knoblauchzehe in Scheiben schneiden. Die Kartoffeln schälen und in Streifen schneiden. Die Schalotten in kleine Stücke schneiden. Die Schalotten und den Knoblauch schälen und in Streifen schneiden. Die Zwiebeln und die Knoblauchzehe in Scheiben schneiden. Die Zwiebel in kleine Stücke schneiden. Die Schalotten in Scheiben schneiden. Die Zwiebel schälen und in Streifen schneiden. Die Schale von den Stielen auf dem Backblech verteilen.  Die Eier mit dem Mixer abschmecken.  Die Schale abschütten. Die Kartoffeln schälen und in Streifen schneiden. Die Schalotten in Streifen schneiden. Die Schalotten in kleine Würfel schneiden. Die Zwiebeln und den Knoblauch schälen und in Stücke schneiden. Die Schalotten in kleine Stücke schneiden. Die Schalotten abziehen u

In [24]:
print('%s' % generate(temperature=0.8,
seed='Alles dazugeben und in einen Kochtopf auf mittlerer Hitze kurz aufkochen lassen.',
                      num_chars=5000))

e einrühren.   Das Fleisch wieder in das klein geschnitten und drei gesauzen weigstück halten und in die Suppe geben. Gelegentlich ausstechen und gleichmäßig aus der Gabel klein schneiden und mit  Deckel schmelzen und abspülen. überschüssige setze wird dann mit Stiel belegen, die Frühlingszwiebeln in beeutketten Teig lassen und mit Salz und Pfeffer würzen. Fleisch darin andünsten und selbe vierteln. sie schälen und fein hacken und in den Bräter geben, mit Kakao bestreuen.  Die Säure das Ei geben und gut miteinander vermischen. dann in den Kühlschrank legen, bei 2150°C ca. 15 Minuten backen.  Tipp: Die Schlagsahne putzen. Die Schokolade mit den Gewürzen bestreuen. Das Basilikum gründlich miteinander verrühren.   Den Feldsalat mit Grundlaufbeeren (man das Rezept fur mit Nussbutter in den Topf geben und es lieber ein mit Backpapier ausgelegten Panderbrot servieren. Die Schalotte in eine Schüssel geben und mit dem Backpulver darüberstreuen, bei 175°C ca. 5 Min. backen.  Das Hackfleisch mit